---
title: "DEDL-HDA-MTG"
subtitle: "This notebook demonstrates how to search, download and raed dta using HDA an satpy."
author: "Serena Avolio (EUMETSAT/Starion)"
tags: [HDA, satpy]
thumbnail: ../../img/EUMETSAT-icon.png
license: MIT
copyright: "© 2025 EUMETSAT"
---
<div style="margin: 6px 0;">
  <a href="https://jupyter.central.data.destination-earth.eu/user-redirect/lab/tree/DestinE-DataLake-Lab/HDA/EUM_data/DEDL-HDA-EO.EUM.DAT.SENTINEL-3.OL_1_ERR___.ipynb" target="_blank" style="text-decoration: none;">
    <span class="launch">🚀 Launch in JupyterHub</span>
  </a>
</div>

# Authenticate

In [ ]:
import destinelab as deauth

In [ ]:
import requests
import json
import os
import zipfile
import datetime
import shutil
from getpass import getpass

In [ ]:
DESP_USERNAME = input("Please input your DESP username or email: ")
DESP_PASSWORD = getpass("Please input your DESP password: ")

auth = deauth.AuthHandler(DESP_USERNAME, DESP_PASSWORD)
access_token = auth.get_token()
if access_token is not None:
    print("DEDL/DESP Access Token Obtained Successfully")
else:
    print("Failed to Obtain DEDL/DESP Access Token")

auth_headers = {"Authorization": f"Bearer {access_token}"}

#  Search

## HDA endpoint
HDA API is based on the Spatio Temporal Asset Catalog specification (STAC), it is convenient define a costant with its endpoint.

In [ ]:
HDA_STAC_ENDPOINT="https://hda.data.destination-earth.eu/stac/v2"

In [ ]:
COLLECTION_ID = "EO.EUM.DAT.MTG.FCI-CLM"
COLLECTION_ID = "EO.EUM.DAT.MTG.FCI-HRFI"

In [ ]:
response = requests.post(HDA_STAC_ENDPOINT+"/search", headers=auth_headers, json={
    "collections": [COLLECTION_ID],
    "datetime": "2025-08-06T08:00:00Z/2025-08-07T00:00:00Z"
})


In [ ]:
from IPython.display import JSON

product = response.json()["features"][0]
JSON(product)

# Download

In [ ]:
from tqdm import tqdm
import time

# Define a list of assets to download
#assets = ["Oa08_radiance.nc", "Oa06_radiance.nc", "Oa02_radiance.nc"]
assets = ["downloadLink"]

for asset in assets:
    download_url = product["assets"][asset]["href"]
    print(download_url)
    filename = asset
    print(filename)
    response = requests.get(download_url, headers=auth_headers)
    total_size = int(response.headers.get("content-length", 0))

    print(f"downloading {filename}")

    with tqdm(total=total_size, unit="B", unit_scale=True) as progress_bar:
        with open(filename, 'wb') as f:
            for data in response.iter_content(1024):
                progress_bar.update(len(data))
                f.write(data)

In [ ]:
zf=zipfile.ZipFile(filename)
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('.')

# Satpy

The Python package satpy supports reading and loading data from many input files. 
For MSG data and the Native format, we can use the satpy reader 'seviri_l1b_native.

In [ ]:
pip install --quiet satpy

Import required libraries

In [ ]:
from satpy.scene import Scene
from satpy.composites import GenericCompositor
from satpy.writers import to_image
from satpy.resample import get_area_def
from satpy import available_readers

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

In [ ]:
#file='W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+FCI-2-CLM--FD------NC4E_C_EUMT_20250806080609_L2PF_OPE_20250806075000_20250806080000_N__C_0048_0000.nc'

### Read and load data

We use the Scene constructor from the satpy library, a Scene object represents a single geographic region of data.
Once loaded we can list all the available bands (spectral channel) for that scene.

In [ ]:
# define path to FCI test data folder
path_to_data = './'
from satpy import find_files_and_readers

In [ ]:
files = find_files_and_readers(base_dir=path_to_data, reader='fci_l1c_nc')

In [ ]:
scn = Scene(filenames=files)

In [ ]:
scn.available_dataset_names()

In [ ]:
scn.available_composite_names()

In [ ]:
scn.load(['night_ir105','vis_06'], upper_right_corner='NE')

In [ ]:
vis_06_values = scn['vis_06'].values

In [ ]:
scn_resampled = scn.resample("eurol", resampler='nearest', radius_of_influence=5000)

In [ ]:
scn_resampled.show('night_ir105')